Overview
========
The code is designed to perform network scanning and port scanning. It discovers devices on a specified network and identifies open ports on these devices. The code uses two primary libraries: Scapy for network scanning and Socket for port scanning.

Functions
=========
scan_network(ip_range)
=
Purpose: This function discovers active devices on the network by sending ARP (Address Resolution Protocol) requests.
Parameters:
ip_range: A string representing the range of IP addresses to scan (e.g., "192.168.1.1/24").
Process:
Create ARP Request: An ARP request is crafted with the destination IP range to find devices.
Create Broadcast Frame: An Ethernet frame with a broadcast MAC address (ff:ff:ff:ff:ff:ff) is created to send the ARP request to all devices on the local network.
Send and Receive Packets: The ARP request packet is sent out, and responses are collected. The srp function from Scapy handles sending and receiving packets.
Process Responses: The function processes the responses to extract the IP and MAC addresses of the devices that replied.
Return Results: A list of dictionaries is returned, each containing the IP and MAC addresses of a detected device.
scan_ports(ip)
=
Purpose: This function checks for open ports on a specified IP address.
Parameters:
ip: A string representing the IP address of the device to scan.
Process:
Scan Ports: The function iterates through a range of common ports (1 to 1024).
Create Socket: For each port, a new socket is created and connected to the IP address and port.
Check Port Status: The connect_ex method is used to try connecting to the port. If the connection is successful (result is 0), the port is considered open.
Close Socket: After each check, the socket is closed to free resources.
Return Results: A list of open ports is returned.
main()
=
Purpose: This function coordinates the network and port scanning processes.
Process:
Define IP Range: Specifies the network range to scan.
Scan Network: Calls scan_network to discover devices on the network.
Print Device Information: Prints the IP and MAC addresses of detected devices.
Scan Ports for Each Device: Calls scan_ports for each detected device to find open ports and prints the results.
Execution: The main function is executed if the script is run directly (if __name__ == "__main__":).
Execution
Network Scanning: The scan_network function detects devices on the network by sending ARP requests and processing the responses.
Port Scanning: The scan_ports function checks for open ports on each detected device by attempting connections to common ports.
Results: The detected devices and their open ports are printed to the console.
This code provides a basic yet effective way to scan a network and identify open ports, which can be useful for network administration, security assessments, or troubleshooting.

In [ ]:
from scapy.all import *
import socket

def scan_network(ip_range):
    # Create an ARP request packet
    arp_request = ARP(pdst=ip_range)
    broadcast = Ether(dst="ff:ff:ff:ff:ff:ff")
    packet = broadcast/arp_request

    # Send the packet and receive responses
    answered_list = srp(packet, timeout=1, verbose=False)[0]

    # Process and print the results
    devices = []
    for element in answered_list:
        device_info = {"ip": element[1].psrc, "mac": element[1].hwsrc}
        devices.append(device_info)
    return devices

def scan_ports(ip):
    open_ports = []
    for port in range(1, 1025):  # Scan ports 1 to 1024
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((ip, port))
        if result == 0:
            open_ports.append(port)
        sock.close()
    return open_ports

# Example usage
ip_range = "192.168.1.1/24"  # Adjust this as needed
devices = scan_network(ip_range)

for device in devices:
    print(f"IP Address: {device['ip']}, MAC Address: {device['mac']}")
    open_ports = scan_ports(device['ip'])
    if open_ports:
        print(f"Open Ports on {device['ip']}: {open_ports}")
    else:
        print(f"No open ports found on {device['ip']}")